# Teams

A team is a group of agents that work together to achieve a common goal.\
I'll first show you how to create and run a team. I’ll then explain how to observe the team’s behavior, which is crucial for debugging and understanding the team’s performance, and common operations to control the team’s behavior.


## Round Robin Group Chat

`RoundRobinGroupChat` is a simple yet effective team configuration where all agents share the same context and take turns responding in a round-robin fashion. Each agent, during its turn, `broadcasts` its response to `all other agents`, ensuring that the entire team maintains a consistent context.\
We will begin by creating a team with two `AssistantAgent` and a `TextMentionTermination` condition that stops the team when a specific word is detected in the agent’s response.\
The two-agent team implements the reflection pattern, a multi-agent design pattern where a critic agent evaluates the responses of a primary agent.

### Basic Example: Generating and Critiquing

In [30]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.base import TaskResult
from autogen_agentchat.conditions import ExternalTermination, TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_core import CancellationToken
from autogen_ext.models.openai import OpenAIChatCompletionClient

import asyncio


In [21]:
# Create an agent that uses the local llm
model_client = OpenAIChatCompletionClient(
    model="ollama/llama3.2:3b",
    api_key="YOUR_API_KEY",
    base_url="http://0.0.0.0:4000/",
    model_capabilities={
        "json_output": False,
        "vision": False,
        "function_calling": True,
        },
        )

#### Create the two agents to work in the team

In [22]:
# Create the primary agent.
primary_agent = AssistantAgent(
    "primary",
    model_client=model_client,
    system_message="You are a helpful AI assistant.",
)

# Create the critic agent.
critic_agent = AssistantAgent(
    "critic",
    model_client=model_client,
    system_message="Provide constructive feedback. Respond with 'APPROVE' only when all your feedbacks are addressed.",
)

# Define a termination condition that stops the task if the critic approves.
text_termination = TextMentionTermination("APPROVE")


#### Create a team with both the agents

In [23]:
# Create a team with the primary and critic agents.
team = RoundRobinGroupChat([primary_agent, critic_agent], termination_condition=text_termination)

#### Run a task for the created team

The team ran the same agent until the termination condition was met. In this case, the termination condition was met when the word “TERMINATE” is detected in the agent’s response.

In [24]:
# Use `asyncio.run(...)` when running in a script.
result = await team.run(task="Give me top 10 achievements of the year 2023.")
print(result)

TaskResult(messages=[TextMessage(source='user', models_usage=None, content='Give me top 10 achievements of the year 2023.', type='TextMessage'), TextMessage(source='primary', models_usage=RequestUsage(prompt_tokens=51, completion_tokens=453), content="As I'm a large language model, I don't have real-time access to current events. However, based on my training data up to December 2023, here are some notable achievements and updates that were reported or made headlines in 2023:\n\n1. **NASA's Artemis Mission**: In February 2023, NASA successfully launched the Orion spacecraft and sent it towards the Moon as part of the Artemis mission, with the goal of returning humans to the lunar surface by 2025.\n2. **First-ever Image of a Black Hole**: The Event Horizon Telescope (EHT) captured the first-ever image of a black hole in April 2023, providing scientists with new insights into these mysterious cosmic objects.\n3. **Quantum Computing Breakthroughs**: Researchers made significant breakthrou

#### Run an task in stream mode in order to observe all the calls by the team

In [ ]:
# When running inside a script, use a async main function and call it from `asyncio.run(...)`.
await team.reset()  # Reset the team for a new task.
async for message in team.run_stream(task="Give me top 10 achievements of the year 2023."):  # type: ignore
    if isinstance(message, TaskResult):
        print("Stop Reason:", message.stop_reason)
    else:
        print(message)

#### Print the streamed calls to the `console`

In [27]:
await team.reset()  # Reset the team for a new task.
await Console(team.run_stream(task="Give me top 10 web searches of the year 2023."))  # Stream the messages to the console.

---------- user ----------
Give me top 10 web searches of the year 2023.
---------- primary ----------
I'd be happy to provide you with some insights on popular web searches for the year 2023. Keep in mind that these are general trends and may not reflect your specific interests or location. Here are the top 10 web searches of 2023, based on publicly available data:

1. **"Artificial Intelligence jobs near me"**: As AI continues to transform industries, people are searching for job opportunities related to this emerging technology.
2. **"Best travel destinations in Europe 2024"**: With travel restrictions easing, many people are planning their next trips and searching for the best places to visit in Europe.
3. **"How to lose weight fast"**: As the new year begins, many individuals search for quick fixes to shed unwanted pounds and start a healthier lifestyle.
4. **"Online courses for data science"**: With the growing demand for data scientists, online courses are becoming increasingly 

TaskResult(messages=[TextMessage(source='user', models_usage=None, content='Give me top 10 web searches of the year 2023.', type='TextMessage'), TextMessage(source='primary', models_usage=RequestUsage(prompt_tokens=52, completion_tokens=433), content='I\'d be happy to provide you with some insights on popular web searches for the year 2023. Keep in mind that these are general trends and may not reflect your specific interests or location. Here are the top 10 web searches of 2023, based on publicly available data:\n\n1. **"Artificial Intelligence jobs near me"**: As AI continues to transform industries, people are searching for job opportunities related to this emerging technology.\n2. **"Best travel destinations in Europe 2024"**: With travel restrictions easing, many people are planning their next trips and searching for the best places to visit in Europe.\n3. **"How to lose weight fast"**: As the new year begins, many individuals search for quick fixes to shed unwanted pounds and sta

#### Stopping and Aborting a team

Apart from automatic termination conditions such as `TextMentionTermination` that stops the team based on the internal state of the team, you can also stop the team from outside by using the `ExternalTermination`.\
Calling `set()` on ExternalTermination will stop the team when the current agent’s turn is over. Thus, the team may not stop immediately. This allows the current agent to finish its turn and broadcast the final message to the team before the team stops, keeping the team’s state consistent.

In [28]:
# Create a new team with an external termination condition.
external_termination = ExternalTermination()
team = RoundRobinGroupChat(
    [primary_agent, critic_agent],
    termination_condition=external_termination | text_termination,  # Use the bitwise OR operator to combine conditions.
)


In [31]:
# Run the team in a background task.
run = asyncio.create_task(Console(team.run_stream(task="Write a short poem about the fall season.")))

# Wait for some time.
await asyncio.sleep(0.1)

# Stop the team.
external_termination.set()

# Wait for the team to finish.
await run

---------- user ----------
Write a short poem about the fall season.
---------- primary ----------
Here's a short poem about the fall season:

As summer's warmth begins to fade,
The trees don vibrant hues of shade,
Golden, crimson, amber bright,
A fleeting glimpse of nature's light.

The air is crisp, the winds do blow,
Leaving whispers of a winter's snow,
The scent of woodsmoke fills the air,
As earthy tones and autumn leaves are shared.

The season's final dance begins,
A whirlwind waltz of colors within,
Nature's canvas, painted with delight,
A final flourish before the winter's night.

So let us bask in fall's warm glow,
And cherish every moment as it grows,
For though the seasons come and go,
Autumn's beauty will forever glow.
[Prompt tokens: 1319, Completion tokens: 150]
---------- Summary ----------
Number of messages: 2
Finish reason: External termination requested
Total prompt tokens: 1319
Total completion tokens: 150
Duration: 5.41 seconds


TaskResult(messages=[TextMessage(source='user', models_usage=None, content='Write a short poem about the fall season.', type='TextMessage'), TextMessage(source='primary', models_usage=RequestUsage(prompt_tokens=1319, completion_tokens=150), content="Here's a short poem about the fall season:\n\nAs summer's warmth begins to fade,\nThe trees don vibrant hues of shade,\nGolden, crimson, amber bright,\nA fleeting glimpse of nature's light.\n\nThe air is crisp, the winds do blow,\nLeaving whispers of a winter's snow,\nThe scent of woodsmoke fills the air,\nAs earthy tones and autumn leaves are shared.\n\nThe season's final dance begins,\nA whirlwind waltz of colors within,\nNature's canvas, painted with delight,\nA final flourish before the winter's night.\n\nSo let us bask in fall's warm glow,\nAnd cherish every moment as it grows,\nFor though the seasons come and go,\nAutumn's beauty will forever glow.", type='TextMessage')], stop_reason='External termination requested')

In [32]:
# Create a cancellation token.
cancellation_token = CancellationToken()

# Use another coroutine to run the team.
run = asyncio.create_task(
    team.run(
        task="Translate the poem to Spanish.",
        cancellation_token=cancellation_token,
    )
)

# Cancel the run.
cancellation_token.cancel()

try:
    result = await run  # This will raise a CancelledError.
except asyncio.CancelledError:
    print("Task was cancelled.")

Task was cancelled.


### Human-in-the-Loop